In [ ]:
# Import the necessary libraries
!pip install spectral 
from sklearn.decomposition import PCA
import os
import scipy.io as sio
import numpy as np
from keras.models import load_model
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score
import itertools
import spectral
import matplotlib
%matplotlib inline
from spectral import *
import spectral.io.envi as envi
from PIL import Image
import pandas as pd
from pickle import load

from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


In [ ]:
# Load the Global values (windowSize, numPCAcomponents, testRatio) from the text file global_variables.txt
myFile = open('/content/drive/My Drive/thesis/global_variables.txt', 'r') 
file = myFile.readlines()[:]


for line in file:

    if line[0:3] == "win":

        ds = line.find('=')
        windowSize = int(line[ds+1:-1],10)

    elif line[0:3] == "num":

        ds = line.find('=')
        numPCAcomponents = int(line[ds+2:-1],10)

    else:

        ds = line.find('=')
        testRatio = float(line[ds+1:])




In [ ]:
def loadDataset():
    image = envi.open('/content/drive/My Drive/thesis/Hyperimage/Reflectance/VNIR/09_VNIR_ROAD_MASKED.hdr',
                      '/content/drive/My Drive/thesis/Hyperimage/Reflectance/VNIR/09_VNIR_ROAD_MASKED')
    data = image.load()
    label_data = Image.open('/content/drive/My Drive/thesis/Hyperimage/Reflectance/VNIR/class4.tif')
    labels = np.array(label_data)
    return data, labels


def reports (X_test,y_test):
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    #target_names = ['Road', 'Pavement', 'Vegetation', 'Train Track', 'Water'
     #          ,'Train Track', 'Shadows']
    target_names = ['Road', 'Vegetation', 'Unclassified', 'Water'
               ,'Train Track']

    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    
    return classification, confusion, Test_Loss, Test_accuracy, kappa


def applyPCA(X, numPCAcomponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numPCAcomponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numPCAcomponents))
    return newX, pca

def Patch(data,height_index,width_index):
    #transpose_array = data.transpose((2,0,1))
    #print transpose_array.shape
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    
    return patch

def standartizeData(X, scaler):
    newX = np.reshape(X, (-1, X.shape[2]))
    newX = scaler.transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1],X.shape[2]))
    return newX

def saveClassifiedData(classes):
    with open("/content/drive/My Drive/thesis/result/radiance/VNIR/training_map_REF_vnir"+ ".npy", 'bw') as outfile:
             np.save(outfile, classes)
def visualize_result(img, predicted_map):
    view = imshow(img, (1, 2, 3), stretch=((0.0, 0.9), (0.0, 0.9), (0.0, 0.9)),figsize=(16, 16), classes=predicted_map)
    view.set_display_mode('overlay')
    view.class_alpha = 1
    view.show_data

def extract_road_edges(road):
    from skimage.feature import canny
    from skimage.viewer import ImageViewer
    from skimage import io
    from skimage import img_as_uint
    edges = canny(
        image=road,
        sigma=5.5,
        low_threshold=0.1,
        high_threshold=0.3,
    )
    #viewer = ImageViewer(edges)
    #viewer.show()
    from skimage.morphology import binary_dilation
    edge = binary_dilation(edges, selem=None, out=None)
    edge = binary_dilation(edge, selem=None, out=None)
    #edge = binary_dilation(edge, selem=None, out=None)
    road_edges = edge.astype(int)
    return road_edges

In [ ]:
X_test = np.load("/content/drive/My Drive/thesis/Hyperimage/Reflectance/VNIR/X_Patches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")
y_test = np.load("/content/drive/My Drive/thesis/Hyperimage/Reflectance/VNIR/y_Patches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")
X_test  = np.reshape(X_test, (X_test.shape[0], X_test.shape[3], X_test.shape[1], X_test.shape[2]))
y_test = np_utils.to_categorical(y_test)
model = load_model('/content/drive/My Drive/thesis/training/my_model1_refVNIR' + str(windowSize) + 'PCA' + str(numPCAcomponents) + "testRatio" + str(testRatio) + '.h5')
scaler = load(open('/content/drive/My Drive/thesis/Hyperimage/Reflectance/VNIR/spatialSize/scaler3.pkl', 'rb'))
X, y = loadDataset()
X,pca = applyPCA(X,numPCAcomponents)
height = y.shape[0]
width = y.shape[1]
PATCH_SIZE = windowSize
numPCAcomponents = numPCAcomponents
outputs = np.zeros((height,width))
for i in range(height-PATCH_SIZE+1):
    for j in range(width-PATCH_SIZE+1):
        image_patch=Patch(X,i,j)
        #print (image_patch.shape)
        X_test_image = image_patch.reshape(1,image_patch.shape[2],image_patch.shape[0],image_patch.shape[1]).astype('float32')                                   
        prediction = (model.predict_classes(X_test_image))                         
        outputs[int(i+PATCH_SIZE/2)][int(j+PATCH_SIZE/2)] = prediction+1

predict_images = imshow(classes = outputs.astype(int),figsize =(16,16))

classes = outputs.astype(int)
visualize_result(X, classes)

saveClassifiedData(classes)

FileNotFoundError: ignored

In [ ]:
val_image = envi.open('/content/drive/My Drive/thesis/Hyperimage/Reflectance/VNIR/validation/08_clippped_road.hdr', '/content/drive/My Drive/thesis/Hyperimage/Reflectance/VNIR/validation/08_clippped_road')
val_img = val_image.load()
val_img.shape

X_val,pca_val = applyPCA(val_img,numPCAcomponents)
X_val = standartizeData(X_val, scaler)

height = X_val.shape[0]
width = X_val.shape[1]
PATCH_SIZE = windowSize
numPCAcomponents = numPCAcomponents
outputs = np.zeros((height,width))
for i in range(height-PATCH_SIZE+1):
    for j in range(width-PATCH_SIZE+1):
        image_patch=Patch(X_val,i,j)
        #print (image_patch.shape)
        X_test_image = image_patch.reshape(1,image_patch.shape[2],image_patch.shape[0],image_patch.shape[1]).astype('float32')                                   
        prediction = (model.predict_classes(X_test_image))                         
        outputs[int(i+PATCH_SIZE/2)][int(j+PATCH_SIZE/2)] = prediction+1

predict_images = imshow(classes = outputs.astype(int),figsize =(16,16))
classes = outputs.astype(int)
visualize_result(X_val, classes)
def saveClassifiedData(classes):
    with open("/content/drive/My Drive/thesis/result/reflectance/VNIR/val_map_REF_vnir11"+ ".npy", 'bw') as outfile:
             np.save(outfile, classes)
saveClassifiedData(classes)

(799, 2296, 186)